# Notkun gervigreindar fyrir teikningu þrívíddarmynda

Nathan HK

In [8]:
import numpy as np
import os
import pywikibot
import sklearn as sk
from sklearn.model_selection import train_test_split
from stl import mesh
import time
import torch
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
torch.set_default_device(device)
mappa = '/Users/002-nathan/Desktop/Envalys/envalys-nathan/'  # Change this

## Inngangsorð
Við ætlum að þjálfa gervigreindarlíkan til að teikna þrívíddarmyndir. Hvernig gerum við það?
- Fyrst reyndi ég að nota þrívíddardíla eða "þríla" (e. *voxels*). Ég taldi punktana í hverjum þríl, og þjálfaði líkan með tölunum, eins og líkan sem greinir ljósmyndir. GitHub: https://github.com/nholmesking/envalys-nathan/blob/4fd89a1515c1c3623a5b8cdf8c766858c465e451/Nathan%20HK.ipynb
  - Vandamál #1: Mac-GPU getur ekki neitt með þrívíddargögnum. Ég þurfti að nota CPU, sem er of hægt.
  - Vandamál #2: Hraðinn er $O(n^3)$.
  - Vandamál #3: Það var engin klár leið til að fara frá greiningu yfir á teikningu.
- Þá reyndi ég að nota líkanið 3DShape2VecSet, sem tekur lista yfir punkta og teinkir myndir sem eru eins og listinn.

Tilvísanir:
- https://arxiv.org/pdf/2301.11445
- https://github.com/1zb/3DShape2VecSet

Ég gerði litlar breytingar á upprunaforritið, því það er eitt forritasafn (e. *module*) sem virkar ekki lengur, og forritið var skrifað fyrir Windows (og ég nota Mac).

## Gögn
Gögnin frá 3DShape2VecSet eru 650 GB samtals; þetta er of mikið fyrir tölvuna mína.

Þessi gögn eru STL-skrár frá Wikimedia Commons. Það eru fimm flokkar:
- líkamshlutar
- byggingar
- rúmfræði
- geimfarartæki
- styttur

### Sækja gögn

In [9]:
flokkar = ['body parts', 'buildings', 'geometric shapes', 'objects in space', 'sculptures']
skrar = {}
catnum = {}

In [10]:
commons = pywikibot.Site('commons', 'commons')
cn = 0
for a in flokkar:
    print(a)
    cat = pywikibot.Category(commons, 'STL files of ' + a)
    catnum[a] = cn
    cn += 1
    n = 0
    for p in cat.members(member_type=['file']):
        if n % 10 == 0:
            print(n)
        mynd = pywikibot.FilePage(p)
        try:
            tempf = open(mappa + 'STLdata/' + a + '_' + p.title()[5:], 'r')
            tempf.close()
        except FileNotFoundError:
            mynd.download(filename=mappa + 'STLdata/' + a + '_' + p.title()[5:])
        try:
            skrar[a].append(p.title()[5:])
        except KeyError:
            skrar[a] = [p.title()[5:]]
        n += 1
        if n >= 100:
            break

body parts
0
10
20
30
40
50
60
70
80
buildings
0
10
20
geometric shapes
0
10
20
30
40
objects in space
0
10
20
30
40
50
sculptures
0
10
20
30
40
50


### Setja upp gögn fyrir notkun
Við eigum að breyta gögnunum úr STL-sniði í sniðið sem 3DShape2VecSet notar. Þetta snið er með lista yfir punkta eftir hvort þeir séu innan á forminu eða utan, en gögnin frá Wikimedia Commons er ekki með það.

Með ```isInside()``` getum útreiknað hvort lína og þríhyrningur mætast, og í hvora átt. Við búum til tvær línur sem fara beint í Z-áttina frá punktnum (eina línu upp og eina línu niður), og teljum þríhyrningana sem línan mætir; ef talan er slétt, þá er punkturinn utan á forminu, og ef talan er oddatala, þá er punkturinn innan. En það tekur of langan tíma; sumar myndir eru með fleiri en milljón þríhyrninga. Þess vegna ritaði ég tvö reiknirit til að gera þetta fljótara:
- Við deilum myndinni í 32x32x32 þríla og merkjum þá sem eru með formspunkta. Fyrir hvern listapunkt: ef þríllinn punktsins er merktur sem formspunktaþríll, þá getum við ekki neitt, en ef hann er ómerktur, þá útreiknum við hvort listapunkturinn sé innan eða utan, og fyrir alla aðra listapunkta sem eru í þessum þríl, við vitum sjálfkrafa. Og við merkjum alla þríla sem eru beint í allar áttir, fram til myndarendarins eða formspunktaþríls, sem innan/utan.
- Við deilum myndinni í 16x16 ílát eftir X- og Y-hnitum (því við leitum að þríhyrningum í Z-áttina). Í byrjun setjum við alla þríhyrninga í þeirra ílát; sumir þríhyrningar, sem eru á línunni milli íláta, eru settir í tvö eða fleiri ílát. Fyrir hvern listapunkt eigum við bara að leita í rétta ílátinu.

Með þessum tveim reikniritum er sniðbreytingin **hundrað** sinnum fljótari. Ég skrifaði sjálfur allt hérna nema ```isInside()```; sjáðu kóðann fyrir tilvísun.

In [11]:
def isInside(a, randpoint):
    if (a[0] > randpoint[0] and a[3] > randpoint[0] and a[6] > randpoint[0]):
        return 0
    if (a[0] < randpoint[0] and a[3] < randpoint[0] and a[6] < randpoint[0]):
        return 0
    if (a[1] > randpoint[1] and a[4] > randpoint[1] and a[7] > randpoint[1]):
        return 0
    if (a[1] < randpoint[1] and a[4] < randpoint[1] and a[7] < randpoint[1]):
        return 0
 
    # This part is from https://diegoinacio.github.io/creative-coding-notebooks-page/pages/ray-intersection_triangle.html
    e = np.array([0, 0, 1])     # Ray direction
    AB = a[3:6] - a[0:3]               # Oriented segment A to B
    AC = a[6:9] - a[0:3]               # Oriented segment A to C
    n = np.cross(AB, AC)     # Normal vector
    n_ = n/np.linalg.norm(n) # Normalized normal
    if not np.any(np.dot(n_, e)):
        return 0
    d = - np.dot(n_, a[0:3])
    # Finding parameter t
    t = - (np.dot(n_, randpoint) + d)/np.dot(n_, e)
    P = randpoint + t*e
    # Get the resulting vector for each vertex
    # following the construction order
    Pa = np.dot(np.cross(a[3:6] - a[0:3], P - a[0:3]), n_)
    Pb = np.dot(np.cross(a[6:9] - a[3:6], P - a[3:6]), n_)
    Pc = np.dot(np.cross(a[0:3] - a[6:9], P - a[6:9]), n_)

    if t > 0 and (Pa > 0 and Pb > 0 and Pc > 0):
        return 1
        
    e = np.array([0, 0, -1])     # Ray direction
    # Finding parameter t
    t = - (np.dot(n_, randpoint) + d)/np.dot(n_, e)
    P = randpoint + t*e
    # Get the resulting vector for each vertex
    # following the construction order
    Pa = np.dot(np.cross(a[3:6] - a[0:3], P - a[0:3]), n_)
    Pb = np.dot(np.cross(a[6:9] - a[3:6], P - a[3:6]), n_)
    Pc = np.dot(np.cross(a[0:3] - a[6:9], P - a[6:9]), n_)

    if t > 0 and (Pa > 0 and Pb > 0 and Pc > 0):
        return -1
    return 0

In [13]:
for cat in skrar:
    tts = []
    print(cat)
    byrjun = time.time()
    os.system('mkdir "' + mappa + '3DShape2VecSet/data/ShapeNetV2_point/' + cat + '"')
    os.system('mkdir "' + mappa + '3DShape2VecSet/data/ShapeNetV2_watertight/' + cat + '"')
    for fi in skrar[cat]:
        gogn = mesh.Mesh.from_file(mappa + 'STLdata/' + cat + '_' + fi)
        utskra_mesh = open(mappa + '3DShape2VecSet/data/ShapeNetV2_watertight/' + cat + '/' + fi + '.npz', 'wb')
        np.savez(utskra_mesh, points=np.array(gogn.v0, dtype=np.float32), scale=1)
        tts.append(fi)
    train, val = train_test_split(tts)
    trainlst = open(mappa + '3DShape2VecSet/data/ShapeNetV2_point/' + cat + '/train.lst', 'w')
    print('train')
    for fi in train:
        print(fi)
        gogn = mesh.Mesh.from_file(mappa + 'STLdata/' + cat + '_' + fi)
        print(gogn.points.shape[0])
        mm = ((min([a[0] for a in gogn.v0]), max([a[0] for a in gogn.v0])),
              (min([a[1] for a in gogn.v0]), max([a[1] for a in gogn.v0])),
              (min([a[2] for a in gogn.v0]), max([a[2] for a in gogn.v0])))
        bucket_id_X = [mm[0][0] + (mm[0][1] - mm[0][0]) * i / 16 for i in range(1, 17)]
        bucket_id_Y = [mm[1][0] + (mm[1][1] - mm[1][0]) * i / 16 for i in range(1, 17)]
        buckets = []
        for i in range(16):
            buckets.append([])
            for j in range(16):
                buckets[i].append([])
        utskra = open(mappa + '3DShape2VecSet/data/ShapeNetV2_point/' + cat + '/' + fi + '.npz', 'wb')
        vol_points = []
        vol_label = []
        scale = (min([mm[0][0], mm[1][0], mm[2][0]]), max([mm[0][1], mm[1][1], mm[2][1]]))
        rescale = lambda x: int((x - scale[0]) * 32 / (scale[1] - scale[0]))
        voxels1 = np.zeros((32, 32, 32), dtype=np.int8)
        voxels2 = np.zeros((32, 32, 32), dtype=np.int8)
        for a in gogn.points:
            p = [0, 0]
            while p[0] < 15 and bucket_id_X[p[0]] < min([a[0], a[3], a[6]]):
                p[0] += 1
                p[1] += 1
            while p[1] < 15 and bucket_id_X[p[1]] < max([a[0], a[3], a[6]]):
                p[1] += 1
            q = [0, 0]
            while q[0] < 15 and bucket_id_Y[q[0]] < min([a[1], a[4], a[7]]):
                q[0] += 1
                q[1] += 1
            while q[1] < 15 and bucket_id_Y[q[1]] < max([a[1], a[4], a[7]]):
                q[1] += 1
            for i in range(p[0], p[1] + 1):
                for j in range(q[0], q[1] + 1):
                    buckets[i][j].append(a)
            for i1 in range(min([rescale(a[0]), rescale(a[3]), rescale(a[6])]), max([rescale(a[0]), rescale(a[3]), rescale(a[6])]) + 1):
                if i1 < 0:
                    i = 0
                elif i1 >= voxels1.shape[0]:
                    i = voxels1.shape[0] - 1
                else:
                    i = i1
                for i2 in range(min([rescale(a[1]), rescale(a[4]), rescale(a[7])]), max([rescale(a[1]), rescale(a[4]), rescale(a[7])]) + 1):
                    if i2 < 0:
                        j = 0
                    elif i2 >= voxels1.shape[0]:
                        j = voxels1.shape[0] - 1
                    else:
                        j = i2
                    for i3 in range(min([rescale(a[2]), rescale(a[5]), rescale(a[8])]), max([rescale(a[2]), rescale(a[5]), rescale(a[8])]) + 1):
                        if i3 < 0:
                            k = 0
                        elif i3 >= voxels1.shape[0]:
                            k = voxels1.shape[0] - 1
                        else:
                            k = i3
                        voxels1[i][j][k] = 1
            n += 1
        for i in range(4096):
            randpoint = (np.random.uniform(mm[0][0], mm[0][1]),
                         np.random.uniform(mm[1][0], mm[1][1]),
                         np.random.uniform(mm[2][0], mm[2][1]))
            # Is point inside or outside shape?
            found = False
            if rescale(randpoint[0]) < 0:
                v0 = 0
            elif rescale(randpoint[0]) >= voxels1.shape[0]:
                v0 = voxels1.shape[0] - 1
            else:
                v0 = rescale(randpoint[0])
            if rescale(randpoint[1]) < 0:
                v1 = 0
            elif rescale(randpoint[1]) >= voxels1.shape[0]:
                v1 = voxels1.shape[0] - 1
            else:
                v1 = rescale(randpoint[1])
            if rescale(randpoint[2]) < 0:
                v2 = 0
            elif rescale(randpoint[2]) >= voxels1.shape[0]:
                v2 = voxels1.shape[0] - 1
            else:
                v2 = rescale(randpoint[2])
            while not found:
                if voxels1[v0][v1][v2] != 0:
                    break
                if voxels1[v0][v1][v2] == 0 and voxels2[v0][v1][v2] != 0:
                    found = True
                    vol_points.append(randpoint)
                    vol_label.append(max(0, voxels2[v0][v1][v2]))
                    break
                if found:
                    break
                v = [v0, v1, v2]
                while v[0] < voxels1.shape[0] and voxels1[v[0]][v1][v2] == 0:
                    if voxels2[v[0]][v1][v2] != 0:
                        found = True
                        vol_points.append(randpoint)
                        vol_label.append(max(0, voxels2[v[0]][v1][v2]))
                        for p in range(v0, v[0]):
                            voxels2[p][v1][v2] = voxels2[v[0]][v1][v2]
                        break
                    v[0] += 1
                if found:
                    break
                v = [v0, v1, v2]
                while v[0] >= 0 and voxels1[v[0]][v1][v2] == 0:
                    if voxels2[v[0]][v1][v2] != 0:
                        found = True
                        vol_points.append(randpoint)
                        vol_label.append(max(0, voxels2[v[0]][v1][v2]))
                        for p in range(v[0] + 1, v0 + 1):
                            voxels2[p][v1][v2] = voxels2[v[0]][v1][v2]
                        break
                    v[0] -= 1
                if found:
                    break
                v = [v0, v1, v2]
                while v[1] < voxels1.shape[1] and voxels1[v0][v[1]][v2] == 0:
                    if voxels2[v0][v[1]][v2] != 0:
                        found = True
                        vol_points.append(randpoint)
                        vol_label.append(max(0, voxels2[v0][v[1]][v2]))
                        for p in range(v1, v[1]):
                            voxels2[v0][p][v2] = voxels2[v0][v[1]][v2]
                        break
                    v[1] += 1
                if found:
                    break
                v = [v0, v1, v2]
                while v[1] >= 0 and voxels1[v0][v[1]][v2] == 0:
                    if voxels2[v0][v[1]][v2] != 0:
                        found = True
                        vol_points.append(randpoint)
                        vol_label.append(max(0, voxels2[v0][v[1]][v2]))
                        for p in range(v[1] + 1, v1 + 1):
                            voxels2[v0][p][v2] = voxels2[v0][v[1]][v2]
                        break
                    v[1] -= 1
                if found:
                    break
                v = [v0, v1, v2]
                while v[2] < voxels1.shape[2] and voxels1[v0][v1][v[2]] == 0:
                    if voxels2[v0][v1][v[2]] != 0:
                        found = True
                        vol_points.append(randpoint)
                        vol_label.append(max(0, voxels2[v0][v1][v[2]]))
                        for p in range(v2, v[2]):
                            voxels2[v0][v1][p] = voxels2[v0][v1][v[2]]
                        break
                    v[2] += 1
                if found:
                    break
                v = [v0, v1, v2]
                while v[2] >= 0 and voxels1[v0][v1][v[2]] == 0:
                    if voxels2[v0][v1][v[2]] != 0:
                        found = True
                        vol_points.append(randpoint)
                        vol_label.append(max(0, voxels2[v0][v1][v[2]]))
                        for p in range(v[2] + 1, v2 + 1):
                            voxels2[v0][v1][p] = voxels2[v0][v1][v[2]]
                        break
                    v[2] -= 1
                break
            if not found:
                fata = [0, 0]
                while fata[0] < 15 and bucket_id_X[fata[0]] < randpoint[0]:
                    fata[0] += 1
                while fata[1] < 15 and bucket_id_Y[fata[1]] < randpoint[1]:
                    fata[1] += 1
                pct = [0, 0]
                for a in buckets[fata[0]][fata[1]]:
                    isi = isInside(a, randpoint)
                    if isi == 1:
                        pct[1] += 1
                    elif isi == -1:
                        pct[0] += 1
                if len(pct) == 2 and pct[0] % 2 == pct[1] % 2:
                    vol_points.append(randpoint)
                    vol_label.append(pct[0] % 2)
                    if pct[0] % 2 == 1:
                        voxels2[v0][v1][v2] = 1
                    else:
                        voxels2[v0][v1][v2] = -1
        print('train vol', time.time() - byrjun)
        near_points = []
        near_label = []
        for i in range(4096):
            randpoint = (np.random.uniform(mm[0][0], mm[0][1]),
                         np.random.uniform(mm[1][0], mm[1][1]),
                         np.random.uniform(mm[2][0], mm[2][1]))
            # Is point inside or outside shape?
            fata = [0, 0]
            while fata[0] < 15 and bucket_id_X[fata[0]] < randpoint[0]:
                fata[0] += 1
            while fata[1] < 15 and bucket_id_Y[fata[1]] < randpoint[1]:
                fata[1] += 1
            pct = [0, 0]
            for a in buckets[fata[0]][fata[1]]:
                isi = isInside(a, randpoint)
                if isi == 1:
                    pct[1] += 1
                elif isi == -1:
                    pct[0] += 1
            if len(pct) == 2 and pct[0] % 2 == pct[1] % 2:
                near_points.append(randpoint)
                near_label.append(pct[0] % 2)
        print('train near', time.time() - byrjun)
        if len(vol_points) > 0 and len(near_points) > 0:
            trainlst.write(fi + '.npz\n')
            np.savez(utskra,
                     vol_points=np.array(vol_points, dtype=np.float32),
                     vol_label=np.array(vol_label),
                     near_points=np.array(near_points, dtype=np.float32),
                     near_label=np.array(near_label))
    trainlst.close()
    vallst = open(mappa + '3DShape2VecSet/data/ShapeNetV2_point/' + cat + '/val.lst', 'w')
    print('val')
    for fi in val:
        print(fi)
        vallst.write(fi + '.npz\n')
        gogn = mesh.Mesh.from_file(mappa + 'STLdata/' + cat + '_' + fi)
        print(gogn.points.shape[0])
        mm = ((min([a[0] for a in gogn.v0]), max([a[0] for a in gogn.v0])),
              (min([a[1] for a in gogn.v0]), max([a[1] for a in gogn.v0])),
              (min([a[2] for a in gogn.v0]), max([a[2] for a in gogn.v0])))
        bucket_id_X = [mm[0][0] + (mm[0][1] - mm[0][0]) * i / 16 for i in range(1, 17)]
        bucket_id_Y = [mm[1][0] + (mm[1][1] - mm[1][0]) * i / 16 for i in range(1, 17)]
        buckets = []
        for i in range(16):
            buckets.append([])
            for j in range(16):
                buckets[i].append([])
        utskra = open(mappa + '3DShape2VecSet/data/ShapeNetV2_point/' + cat + '/' + fi + '.npz', 'wb')
        vol_points = []
        vol_label = []
        scale = (min([mm[0][0], mm[1][0], mm[2][0]]), max([mm[0][1], mm[1][1], mm[2][1]]))
        rescale = lambda x: int((x - scale[0]) * 32 / (scale[1] - scale[0]))
        voxels1 = np.zeros((32, 32, 32), dtype=np.int8)
        voxels2 = np.zeros((32, 32, 32), dtype=np.int8)
        for a in gogn.points:
            p = [0, 0]
            while p[0] < 15 and bucket_id_X[p[0]] < min([a[0], a[3], a[6]]):
                p[0] += 1
                p[1] += 1
            while p[1] < 15 and bucket_id_X[p[1]] < max([a[0], a[3], a[6]]):
                p[1] += 1
            q = [0, 0]
            while q[0] < 15 and bucket_id_Y[q[0]] < min([a[1], a[4], a[7]]):
                q[0] += 1
                q[1] += 1
            while q[1] < 15 and bucket_id_Y[q[1]] < max([a[1], a[4], a[7]]):
                q[1] += 1
            for i in range(p[0], p[1] + 1):
                for j in range(q[0], q[1] + 1):
                    buckets[i][j].append(a)
            for i1 in range(min([rescale(a[0]), rescale(a[3]), rescale(a[6])]), max([rescale(a[0]), rescale(a[3]), rescale(a[6])]) + 1):
                if i1 < 0:
                    i = 0
                elif i1 >= voxels1.shape[0]:
                    i = voxels1.shape[0] - 1
                else:
                    i = i1
                for i2 in range(min([rescale(a[1]), rescale(a[4]), rescale(a[7])]), max([rescale(a[1]), rescale(a[4]), rescale(a[7])]) + 1):
                    if i2 < 0:
                        j = 0
                    elif i2 >= voxels1.shape[0]:
                        j = voxels1.shape[0] - 1
                    else:
                        j = i2
                    for i3 in range(min([rescale(a[2]), rescale(a[5]), rescale(a[8])]), max([rescale(a[2]), rescale(a[5]), rescale(a[8])]) + 1):
                        if i3 < 0:
                            k = 0
                        elif i3 >= voxels1.shape[0]:
                            k = voxels1.shape[0] - 1
                        else:
                            k = i3
                        voxels1[i][j][k] = 1
        for i in range(4096):
            randpoint = (np.random.uniform(mm[0][0], mm[0][1]),
                         np.random.uniform(mm[1][0], mm[1][1]),
                         np.random.uniform(mm[2][0], mm[2][1]))
            # Is point inside or outside shape?
            found = False
            if rescale(randpoint[0]) < 0:
                v0 = 0
            elif rescale(randpoint[0]) >= voxels1.shape[0]:
                v0 = voxels1.shape[0] - 1
            else:
                v0 = rescale(randpoint[0])
            if rescale(randpoint[1]) < 0:
                v1 = 0
            elif rescale(randpoint[1]) >= voxels1.shape[0]:
                v1 = voxels1.shape[0] - 1
            else:
                v1 = rescale(randpoint[1])
            if rescale(randpoint[2]) < 0:
                v2 = 0
            elif rescale(randpoint[2]) >= voxels1.shape[0]:
                v2 = voxels1.shape[0] - 1
            else:
                v2 = rescale(randpoint[2])
            while not found:
                if voxels1[v0][v1][v2] != 0:
                    break
                if voxels1[v0][v1][v2] == 0 and voxels2[v0][v1][v2] != 0:
                    found = True
                    vol_points.append(randpoint)
                    vol_label.append(max(0, voxels2[v0][v1][v2]))
                    break
                if found:
                    break
                v = [v0, v1, v2]
                while v[0] < voxels1.shape[0] and voxels1[v[0]][v1][v2] == 0:
                    if voxels2[v[0]][v1][v2] != 0:
                        found = True
                        vol_points.append(randpoint)
                        vol_label.append(max(0, voxels2[v[0]][v1][v2]))
                        for p in range(v0, v[0]):
                            voxels2[p][v1][v2] = voxels2[v[0]][v1][v2]
                        break
                    v[0] += 1
                if found:
                    break
                v = [v0, v1, v2]
                while v[0] >= 0 and voxels1[v[0]][v1][v2] == 0:
                    if voxels2[v[0]][v1][v2] != 0:
                        found = True
                        vol_points.append(randpoint)
                        vol_label.append(max(0, voxels2[v[0]][v1][v2]))
                        for p in range(v[0] + 1, v0 + 1):
                            voxels2[p][v1][v2] = voxels2[v[0]][v1][v2]
                        break
                    v[0] -= 1
                if found:
                    break
                v = [v0, v1, v2]
                while v[1] < voxels1.shape[1] and voxels1[v0][v[1]][v2] == 0:
                    if voxels2[v0][v[1]][v2] != 0:
                        found = True
                        vol_points.append(randpoint)
                        vol_label.append(max(0, voxels2[v0][v[1]][v2]))
                        for p in range(v1, v[1]):
                            voxels2[v0][p][v2] = voxels2[v0][v[1]][v2]
                        break
                    v[1] += 1
                if found:
                    break
                v = [v0, v1, v2]
                while v[1] >= 0 and voxels1[v0][v[1]][v2] == 0:
                    if voxels2[v0][v[1]][v2] != 0:
                        found = True
                        vol_points.append(randpoint)
                        vol_label.append(max(0, voxels2[v0][v[1]][v2]))
                        for p in range(v[1] + 1, v1 + 1):
                            voxels2[v0][p][v2] = voxels2[v0][v[1]][v2]
                        break
                    v[1] -= 1
                if found:
                    break
                v = [v0, v1, v2]
                while v[2] < voxels1.shape[2] and voxels1[v0][v1][v[2]] == 0:
                    if voxels2[v0][v1][v[2]] != 0:
                        found = True
                        vol_points.append(randpoint)
                        vol_label.append(max(0, voxels2[v0][v1][v[2]]))
                        for p in range(v2, v[2]):
                            voxels2[v0][v1][p] = voxels2[v0][v1][v[2]]
                        break
                    v[2] += 1
                if found:
                    break
                v = [v0, v1, v2]
                while v[2] >= 0 and voxels1[v0][v1][v[2]] == 0:
                    if voxels2[v0][v1][v[2]] != 0:
                        found = True
                        vol_points.append(randpoint)
                        vol_label.append(max(0, voxels2[v0][v1][v[2]]))
                        for p in range(v[2] + 1, v2 + 1):
                            voxels2[v0][v1][p] = voxels2[v0][v1][v[2]]
                        break
                    v[2] -= 1
                break
            if not found:
                fata = [0, 0]
                while fata[0] < 15 and bucket_id_X[fata[0]] < randpoint[0]:
                    fata[0] += 1
                while fata[1] < 15 and bucket_id_Y[fata[1]] < randpoint[1]:
                    fata[1] += 1
                pct = [0, 0]
                for a in buckets[fata[0]][fata[1]]:
                    isi = isInside(a, randpoint)
                    if isi == 1:
                        pct[1] += 1
                    elif isi == -1:
                        pct[0] += 1
            if len(pct) == 2 and pct[0] % 2 == pct[1] % 2:
                vol_points.append(randpoint)
                vol_label.append(pct[0] % 2)
        if len(vol_points) > 0:
            print('val vol', time.time() - byrjun)
            np.savez(utskra,
                     vol_points=np.array(vol_points, dtype=np.float32),
                     vol_label=np.array(vol_label))
    vallst.close()
    print(time.time() - byrjun)
    print('----')

body parts


mkdir: /Users/002-nathan/Desktop/Envalys/envalys-nathan/3DShape2VecSet/data/ShapeNetV2_point/body parts: File exists
mkdir: /Users/002-nathan/Desktop/Envalys/envalys-nathan/3DShape2VecSet/data/ShapeNetV2_watertight/body parts: File exists


train
We Are Beautiful – 591522-FSAN-1 – surface.stl
521873
train vol 30.76405096054077
train near 54.7297248840332
We Are Beautiful – 426060-FSAN-1 – surface.stl
244908
train vol 70.11481285095215
train near 82.91095399856567
We Are Beautiful – 591522-FSAN-2 – solid.stl
386810
train vol 103.04053568840027
train near 121.73288702964783
Vh-m-larynx.stl
13238
train vol 123.69002485275269
train near 126.53056001663208
Vh-f-ovary-l.stl
424
train vol 127.47269773483276
train near 128.9628930091858
Vh-m-main-bronchus.stl
6023
train vol 129.96245193481445
train near 131.47602200508118
Vh-m-blood-vasculature.stl
416538
train vol 152.09684777259827
train near 171.2046856880188
We Are Beautiful – 320912-FSAN-1 – surface.stl
478264
train vol 197.48278975486755
train near 220.02212190628052
We Are Beautiful – 591522-FSAN-5 – surface.stl
371164
train vol 241.98857069015503
train near 260.70286107063293
We Are Beautiful – 591522-FSAN-3 – surface.stl
130469
train vol 269.15755796432495
train near 276

  n_ = n/np.linalg.norm(n) # Normalized normal



val vol 1846.5825469493866
Female lymph node right.stl
254663
val vol 1880.4115147590637
Vh-f-mammary-gland-r.stl
585330
val vol 1926.5773029327393
We Are Beautiful – 628398-FSAN-1 – solid.stl
235046
val vol 1946.271229982376
Hollow face illusion.stl
2518
val vol 1947.992458820343
We Are Beautiful – 662767-FSAN-1 – solid.stl
487252
val vol 1987.5273818969727
Vh-m-kidney-l.stl
83692
val vol 2004.201350927353
Vh-f-fallopian-tube-r.stl
17770
val vol 2006.9491877555847
We Are Beautiful – 591522-FSAN-4 – solid.stl
414088
val vol 2041.7979607582092
Vh-f-main-bronchus.stl
33865
val vol 2046.638457775116
We Are Beautiful – 693514-FSAN-1 – solid.stl
108116
val vol 2057.282483816147
We Are Beautiful – 755536-FSAN-1 – solid.stl
954586
val vol 2130.7333698272705
3D partial human mandible with teeth.stl
309098
val vol 2156.143615961075
Vh-m-eye-l.stl
346630
val vol 2213.0813839435577
Reconstrução facial - busto - Cáceres-MT.stl
199966
val vol 2231.923610687256
3D foot.stl
205676
val vol 2252.124358

...

train
Inverse-reuleaux-tetrahedron-3-intersection.stl
5786
train vol 2.999013900756836
train near 4.540855646133423
2 subdivisions geodome made from half of Blender isosphere.stl
40
train vol 5.517729759216309
train near 6.315144777297974
Earth.stl
122880
train vol 13.111702680587769
train near 21.33675980567932
LowPoly Sphere 07.stl
52
train vol 22.65981674194336
train near 24.223251819610596
Inverse-reuleaux-tetrahedron-2-intersection.stl
11158
train vol 26.043884754180908
train near 27.86289668083191
Goldberg polyhedron with equal edge lengths.stl
636
train vol 29.04257583618164
train near 31.120158910751343
LowPoly Sphere 20.stl
396
train vol 32.231242656707764
train near 34.09234881401062
LowPoly Sphere 10.stl
96
train vol 35.38839769363403
train near 37.10867667198181
Twisted torus.stl
13824
train vol 39.93601179122925
train near 42.945066928863525
LowPoly Sphere 05.stl
26
train vol 44.458961725234985
train near 46.01227688789368
Point selle.stl
2628
train vol 46.527501821517944


mkdir: /Users/002-nathan/Desktop/Envalys/envalys-nathan/3DShape2VecSet/data/ShapeNetV2_point/objects in space: File exists
mkdir: /Users/002-nathan/Desktop/Envalys/envalys-nathan/3DShape2VecSet/data/ShapeNetV2_watertight/objects in space: File exists


exception (False, 'No lines found, impossible to read')
train
AIM-composite-full.stl
132110
train vol 13.264106035232544
train near 23.785359144210815
Aura 2013.stl
115781
train vol 34.65330624580383
train near 46.49265003204346
WFIRST model.stl
544774
train vol 102.80066108703613
train near 165.6384162902832
CNOFS-comp-2012-1.stl
75903
train vol 169.1447331905365
train near 172.6557011604309
Aqua-Composite-Full.stl
110831
train vol 182.41203713417053
train near 191.36736416816711
Chandra v09.stl
30132
train vol 194.7120611667633
train near 197.73111510276794
SAOCOM HORIZONTAL calado 1 sin circulos REP.stl
1362060
train vol 259.57711720466614
train near 318.2503662109375
CYGNSS solo 39.stl
7338
train vol 319.9399471282959
train near 321.3176922798157
New Horizons.stl
160886
train vol 331.8513331413269
train near 344.6298623085022
Deep Space 1 11 3D Model.stl
13812
train vol 346.3556983470917
train near 348.09108710289
Aqua 13 3D model.stl
49415
train vol 351.5558819770813
train near 35

mkdir: /Users/002-nathan/Desktop/Envalys/envalys-nathan/3DShape2VecSet/data/ShapeNetV2_point/sculptures: File exists
mkdir: /Users/002-nathan/Desktop/Envalys/envalys-nathan/3DShape2VecSet/data/ShapeNetV2_watertight/sculptures: File exists


train
十九夜塔（古河市新久田）.stl
999978
train vol 66.44608736038208
train near 112.67674541473389
Pandora-James Pradier-MAHG Inv 1991-0006-High poly.stl
433057
train vol 143.0450313091278
train near 167.82599425315857
御嶽塔（天王社）.stl
1474128
train vol 259.0602490901947
train near 335.1171340942383
五神名地神塔（横浜市旭区）.stl
1473158
train vol 425.5075042247772
train near 518.7826943397522
馬頭観音（ごはん塚）.stl
1500000
train vol 606.630045413971
train near 673.61377120018
Asad Al-Lat.stl
29404
train vol 677.5948894023895
train near 682.1095342636108
二十三夜塔（印西市大森）.stl
1246500
train vol 741.8338570594788
train near 801.1162071228027
Scan the World - Venus of Willendorf.stl
699996
train vol 838.1284601688385
train near 873.3385012149811
庚申塔道標（横浜市旭区今宿南町）.stl
1750000
train vol 967.0167243480682
train near 1044.0845482349396
Scan the World - Belvedere Torso.stl
97796
train vol 1055.8446531295776
train near 1075.0992093086243
Thingiverse - Bastet.stl
16524
train vol 1078.713355064392
train near 1082.3284921646118
廿三夜塔（横浜市都筑

Sniðbreytingin tók nærri 4 klst.

## Líkan
Líkanið þarf ekki að búa til góðar myndir. Það þarf bara að virka án villna, sem sýnir okkur að við þurfum bara meiri og betri gögn, því vísindagreinin er með góðar myndir.

In [18]:
os.system('cd ' + mappa)
os.system('torchrun --nproc_per_node=4 3DShape2VecSet/main_ae.py --accum_iter=2 --model ae_d512_m512 '
          '--data_path 3DShape2VecSet/data --output_dir output/ae/ae_d512_m512 --log_dir output/ae/ae_d512_m512 '
          '--num_workers 10 --point_cloud_size 2048 --batch_size 64 --epochs 200 --warmup_epochs 5')

W0825 09:05:21.680789 8286703424 torch/distributed/elastic/multiprocessing/redirects.py:28] NOTE: Redirects are currently not supported in Windows or MacOs.
W0825 09:05:21.693071 8286703424 torch/distributed/run.py:779] 
W0825 09:05:21.693071 8286703424 torch/distributed/run.py:779] *****************************************
W0825 09:05:21.693071 8286703424 torch/distributed/run.py:779] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
W0825 09:05:21.693071 8286703424 torch/distributed/run.py:779] *****************************************


['body parts', 'buildings', 'geometric shapes', 'objects in space', 'sculptures']
['body parts', 'buildings', 'geometric shapes', 'objects in space', 'sculptures']['body parts', 'buildings', 'geometric shapes', 'objects in space', 'sculptures']

['body parts', 'buildings', 'geometric shapes', 'objects in space', 'sculptures']
['body parts', 'buildings', 'geometric shapes', 'objects in space', 'sculptures']
['body parts', 'buildings', 'geometric shapes', 'objects in space', 'sculptures']
['body parts', 'buildings', 'geometric shapes', 'objects in space', 'sculptures']
['body parts', 'buildings', 'geometric shapes', 'objects in space', 'sculptures']
Model = AutoEncoder(
  (cross_attend_blocks): ModuleList(
    (0): PreNorm(
      (fn): Attention(
        (to_q): Linear(in_features=512, out_features=512, bias=False)
        (to_kv): Linear(in_features=512, out_features=1024, bias=False)
        (to_out): Linear(in_features=512, out_features=512, bias=True)
        (drop_path): Identity()


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/amp/grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/amp/grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/amp/grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/amp/grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


Epoch: [0]  [ 0/47]  eta: 0:43:45  lr: 0.000000  loss: 0.7250 (0.7250)  loss_vol: 0.6592 (0.6592)  loss_near: 0.6580 (0.6580)  iou: 0.0139 (0.0139)  time: 55.8586  data: 19.8939
Epoch: [0]  [ 0/47]  eta: 0:49:22  lr: 0.000000  loss: 0.7185 (0.7185)  loss_vol: 0.6532 (0.6532)  loss_near: 0.6534 (0.6534)  iou: 0.0089 (0.0089)  time: 63.0310  data: 19.9302
Epoch: [0]  [ 0/47]  eta: 0:50:54  lr: 0.000000  loss: 0.7204 (0.7204)  loss_vol: 0.6549 (0.6549)  loss_near: 0.6550 (0.6550)  iou: 0.0234 (0.0234)  time: 64.9863  data: 16.6555
Epoch: [0]  [ 0/47]  eta: 1:23:32  lr: 0.000000  loss: 0.7272 (0.7272)  loss_vol: 0.6611 (0.6611)  loss_near: 0.6610 (0.6610)  iou: 0.0238 (0.0238)  time: 106.6386  data: 16.7561


W0825 09:35:12.375863 8286703424 torch/distributed/elastic/agent/server/api.py:688] Received Signals.SIGINT death signal, shutting down workers
W0825 09:35:12.392325 8286703424 torch/distributed/elastic/multiprocessing/api.py:858] Sending process 40348 closing signal SIGINT
W0825 09:35:12.393383 8286703424 torch/distributed/elastic/multiprocessing/api.py:858] Sending process 40349 closing signal SIGINT
W0825 09:35:12.393764 8286703424 torch/distributed/elastic/multiprocessing/api.py:858] Sending process 40350 closing signal SIGINT
W0825 09:35:12.398679 8286703424 torch/distributed/elastic/multiprocessing/api.py:858] Sending process 40351 closing signal SIGINT
W0825 09:35:42.408644 8286703424 torch/distributed/elastic/multiprocessing/api.py:875] Unable to shutdown process 40348 via Signals.SIGINT, forcefully exiting via Signals.SIGKILL
W0825 09:35:49.303166 8286703424 torch/distributed/elastic/multiprocessing/api.py:875] Unable to shutdown process 40349 via Signals.SIGINT, forcefully ex

256

Það eru 200 lotna (e. *epochs*), og hver lota tekur klukkustund; við eigum ekki nógan tíma fyrir alla þjálfunina. **En hún er hafin án villna.**

# Lokaorð
Hvert förum við héðan? 3DShape2VecSet er með margar takmarkanir.
- Það kann bara að búa til einfaldar myndir, eins og "stóll". Með greiningarlíkani getum við mögulega búið til fjölfaldar myndir. Ef notandi segir "stóll með kodda", þá býr líkanið okkar til mynd af stóli og mynd af kodda, sem líkanið sameinar.
- Það skilur ekki mannamál.
- Það er ekki með mikil gögn. Með mínu forriti hérna getum við búið til eins mikil gögn og við viljum; við getum tekið myndir frá notendum og búum til lista yfir punkta, og notað þá til að þjálfa líkanið.

Ég er viss um að, með meiri gögn og betri tölvu, við getum að minnsta kosti eitt af þessum.